In [195]:
import pypsa
import pandas as pd
import logging

In [115]:
def create_network(capacity):
    
    max_capacity = [np.inf if unit==0 else unit for unit in capacity]
    network = pypsa.Network()

    # Create an empty network.
    network.set_snapshots(range(2))

    # Add a bus to the network.
    network.add("Bus","bus")

    # Add a coal power plant to the bus.
    network.add("Generator","coal",bus="bus",
           committable=True,
           p_min_pu=0.2,
           marginal_cost=15,
           capital_cost= 100,
           p_nom_extendable=True,
           p_nom_min=capacity[0],
           p_nom_max=max_capacity[0])

    # Add a gas power plant to the bus.
    network.add("Generator","gas",bus="bus",
           committable=True,
           marginal_cost=20,
           capital_cost=75,
           p_min_pu=0,
           p_nom_extendable=True,
           p_nom_min=capacity[1],
           p_nom_max=max_capacity[1])

    # Add a 2nd gas power plant to the bus.
    network.add("Generator","bio",bus="bus",
           committable=True,
           marginal_cost=18,
           capital_cost = 115,
           p_min_pu=0,
           p_nom_extendable=True,
           p_nom_min=capacity[2],
           p_nom_max=max_capacity[2])

    # Add load to the bus.
    network.add("Load","load",bus="bus",p_set=[6000,1200])

    return network

In [130]:
network = create_network([0,0,0])

In [131]:
# Initial solution of network, with non MGA objective function 
network.lopf(network.snapshots,solver_name='gurobi')
# Saving the value of the old objective function.
old_objective_value = network.model.objective()
print('old objective valeu: ' + str(old_objective_value))

INFO:pypsa.pf:Slack bus for sub-network 0 is bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
Index(['coal', 'gas', 'bio'], dtype='object')
Currently PyPSA cannot do both these functions, so PyPSA is choosing investment optimisation for these generators.
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x12
  Lower bound: 594000.0
  Upper bound: 594000.0
  Number of objectives: 1
  Number of constraints: 17
  Number of variables: 12
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 12
  Number of nonzeros: 29
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality 

### Adding MGA constraint 

In [132]:
def constraint_functionality(network, snapshots,  MGA_slack = 0.05):
    import pyomo.environ as pyomo_env
    MGA_slack = 0.1

    network.model.mga_constraint = pyomo_env.Constraint(expr=network.model.objective.expr <= 
                                                  (1 + MGA_slack) * old_objective_value)

    print((1 + MGA_slack) * old_objective_value)

In [134]:
def update_network(network,capacity):
    
    max_capacity = [np.inf if unit==0 else unit for unit in capacity]
    
    network.generators.at['coal','p_nom_min'] = capacity[0]
    network.generators.at['gas','p_nom_min'] = capacity[1]
    network.generators.at['bio','p_nom_min'] = capacity[2]
    
    network.generators.at['coal','p_nom_max'] = max_capacity[0]
    network.generators.at['gas','p_nom_max'] = max_capacity[1]
    network.generators.at['bio','p_nom_max'] = max_capacity[2]
    
    return network

In [203]:
def test_point(network,variables):

    pypsa.pf.logger.setLevel(logging.ERROR)
    network = update_network(network,variables)
    
    out = network.lopf(network.snapshots,\
                 solver_name='gurobi',\
                 skip_pre=True,\
                 extra_functionality=lambda network,\
                 snapshots: constraint_functionality(network, snapshots, MGA_slack))

    if str(network.results['Solver'].status) == 'ok':
        feasable = True
    else : 
        feasable = False
        
    variables_out = list(network.generators.p_nom_opt)

    return(feasable,variables_out)
    

### Finding max and min values 

In [136]:
import pyomo.environ as pyomo_env
# Defining exstra functionality, that updates the objective function of the network
def maxmin_functionality(network, snapshots,  MGA_slack = 0.05, counter = 0):
    # Identify the nonzero decision variables that should enter the MGA objective function.
    
    MGA_objective = 0
    variables = [gen_p for gen_p in network.model.generator_p_nom]
    gen_p = variables[int(counter/2)]
    MGA_objective += network.model.generator_p_nom[gen_p]
    if counter%2 == 0 :
        MGA_objective = -MGA_objective
    print(MGA_objective)

    # Add the new MGA objective function to the model.
    network.model.mga_objective = pyomo_env.Objective(expr=MGA_objective)
    # Deactivate the old objective function and activate the MGA objective function.
    network.model.objective.deactivate()
    network.model.mga_objective.activate()
    # Add the MGA slack constraint.
    #print('old objective value ',old_objective_value)
    network.model.mga_constraint = pyomo_env.Constraint(expr=network.model.objective.expr <= 
                                          (1 + MGA_slack) * old_objective_value)

In [137]:
counter = 1
MGA_slack = 0.1

network= update_network(network,[0,0,0])

network.solutions = []

for counter in range(len(network.generators)*2):
    print(counter)
    network.lopf(network.snapshots,\
                         solver_name='gurobi',\
                         extra_functionality=lambda network,\
                         snapshots: maxmin_functionality(network, snapshots, MGA_slack,counter))
    network.solutions.append(network.copy())

INFO:pypsa.pf:Slack bus for sub-network 0 is bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
Index(['coal', 'gas', 'bio'], dtype='object')
Currently PyPSA cannot do both these functions, so PyPSA is choosing investment optimisation for these generators.
INFO:pypsa.opf:Solving model using gurobi


0
- generator_p_nom[coal]


INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x12
  Lower bound: -3270.0
  Upper bound: -3270.0
  Number of objectives: 1
  Number of constraints: 18
  Number of variables: 12
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 12
  Number of nonzeros: 38
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (s

INFO:pypsa.pf:Slack bus for sub-network 0 is bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
Index(['coal', 'gas', 'bio'], dtype='object')
Currently PyPSA cannot do both these functions, so PyPSA is choosing investment optimisation for these generators.
INFO:pypsa.opf:Solving model using gurobi


...

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x12
  Lower bound: -1626.3157894736837
  Upper bound: -1626.3157894736837
  Number of objectives: 1
  Number of constraints: 18
  Number of variables: 12
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 12
  Number of nonzeros: 38
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was

INFO:pypsa.pf:Slack bus for sub-network 0 is bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
Index(['coal', 'gas', 'bio'], dtype='object')
Currently PyPSA cannot do both these functions, so PyPSA is choosing investment optimisation for these generators.
INFO:pypsa.opf:Solving model using gurobi


5
generator_p_nom[bio]


INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x12
  Lower bound: 0.0
  Upper bound: 0.0
  Number of objectives: 1
  Number of constraints: 18
  Number of variables: 12
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 12
  Number of nonzeros: 38
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject t

In [138]:

coal_max = (float(network.solutions[0].generators.loc['coal'].p_nom_opt))
coal_min = (float(network.solutions[1].generators.loc['coal'].p_nom_opt))
gas_max =(float(network.solutions[2].generators.loc['gas'].p_nom_opt))
gas_min = (float(network.solutions[3].generators.loc['gas'].p_nom_opt))
bio_max = (float(network.solutions[4].generators.loc['bio'].p_nom_opt))
bio_min = (float(network.solutions[5].generators.loc['bio'].p_nom_opt))

In [139]:
for i in range(len(network.solutions)):
    print(network.solutions[i].generators_t.p)

     coal     gas  bio
0  3270.0  2730.0  0.0
1  1200.0     0.0  0.0
   coal     gas  bio
0   0.0  6000.0  0.0
1   0.0  1200.0  0.0
   coal     gas  bio
0   0.0  6000.0  0.0
1   0.0  1200.0  0.0
     coal     gas  bio
0  3270.0  2730.0  0.0
1  1200.0     0.0  0.0
   coal          gas          bio
0   0.0  4373.684211  1626.315789
1   0.0     0.000000  1200.000000
     coal     gas  bio
0  3270.0  2730.0  0.0
1  1200.0     0.0  0.0


In [144]:
network.solutions[2].generators.p_nom_opt

coal       0.0
gas     6792.0
bio        0.0
Name: p_nom_opt, dtype: float64

In [200]:
class grid :
    pass

dens = 20
grid.coal,grid.gas  = np.meshgrid(np.linspace(coal_min,coal_max,dens),np.linspace(gas_min,gas_max,dens))

grid.coal = grid.coal.reshape(dens**2)
grid.gas = grid.gas.reshape(dens**2)
#y2 = y2.reshape(11**2)

In [201]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot defining corner points
ax.plot(grid.coal, grid.gas, grid.gas, "ko")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [205]:
test = []
grid.test = []
for coal,gas in zip(grid.coal,grid.gas):
    is_good , variables_out = test_point(network,[coal,gas,0])
    test.append(is_good)
    grid.test.append(variables_out)

INFO:pypsa.opf:Building pyomo model using `angles` formulation
Index(['coal', 'gas', 'bio'], dtype='object')
Currently PyPSA cannot do both these functions, so PyPSA is choosing investment optimisation for these generators.
INFO:pypsa.opf:Solving model using gurobi


653400.0


INFO:pypsa.opf:Optimization successful
INFO:pypsa.opf:Building pyomo model using `angles` formulation
Index(['coal', 'gas', 'bio'], dtype='object')
Currently PyPSA cannot do both these functions, so PyPSA is choosing investment optimisation for these generators.
INFO:pypsa.opf:Solving model using gurobi


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x12
  Lower bound: 653400.0
  Upper bound: 653400.0
  Number of objectives: 1
  Number of constraints: 18
  Number of variables: 12
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 12
  Number of nonzeros: 38
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality 

    message from solver=Problem proven to be infeasible or unbounded.
ERROR:pypsa.opf:Optimisation failed with status warning and terminal condition infeasibleOrUnbounded
INFO:pypsa.opf:Building pyomo model using `angles` formulation
Index(['coal', 'gas', 'bio'], dtype='object')
Currently PyPSA cannot do both these functions, so PyPSA is choosing investment optimisation for these generators.
INFO:pypsa.opf:Solving model using gurobi


...

    model=Linear Optimal Power Flow;
        message from solver=Problem proven to be infeasible or unbounded.
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x12
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 18
  Number of variables: 12
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 12
  Number of nonzeros: 38
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: warning
  Return code: 0
  Message: Problem proven to be infeasible or unbounded.
  Termination condition: inf

    message from solver=Problem proven to be infeasible or unbounded.
ERROR:pypsa.opf:Optimisation failed with status warning and terminal condition infeasibleOrUnbounded
INFO:pypsa.opf:Building pyomo model using `angles` formulation
Index(['coal', 'gas', 'bio'], dtype='object')
Currently PyPSA cannot do both these functions, so PyPSA is choosing investment optimisation for these generators.
INFO:pypsa.opf:Solving model using gurobi


    model=Linear Optimal Power Flow;
        message from solver=Problem proven to be infeasible or unbounded.
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x12
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 18
  Number of variables: 12
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 12
  Number of nonzeros: 38
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: warning
  Return code: 0
  Message: Problem proven to be infeasible or unbounded.
  Termination condition: inf

    message from solver=Problem proven to be infeasible or unbounded.
ERROR:pypsa.opf:Optimisation failed with status warning and terminal condition infeasibleOrUnbounded


    model=Linear Optimal Power Flow;
        message from solver=Problem proven to be infeasible or unbounded.
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x12
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 18
  Number of variables: 12
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 12
  Number of nonzeros: 38
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: warning
  Return code: 0
  Message: Problem proven to be infeasible or unbounded.
  Termination condition: inf

In [206]:
%matplotlib widget
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for is_blue,i in zip(test,range(len(test))):
    if is_blue:
        ax.plot([grid.test[i][0]], [grid.test[i][1]], [grid.test[i][2]], "bo")
    #else :
        #ax.plot([grid.test[i][0]], [grid.test[i][1]], [grid.test[i][2]], "ro")
# Make axis label
for i in ["x", "y", "z"]:
    eval("ax.set_{:s}label('{:s}')".format(i, i))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [233]:
from scipy.spatial import ConvexHull, convex_hull_plot_2d
points = np.array(grid.test)
hull = ConvexHull(points)



In [234]:
hull.volume

659873445.680128

In [236]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot defining corner points
ax.plot(points.T[0], points.T[1], points.T[2], "ko")

# 12 = 2 * 6 faces are the simplices (2 simplices per square face)
for s in hull.simplices:
    
    s = np.append(s, s[0])  # Here we cycle back to the first coordinate
    print(s)
    ax.plot(points[s, 0], points[s, 1], points[s, 2], "r-")

# Make axis label
for i in ["x", "y", "z"]:
    eval("ax.set_{:s}label('{:s}')".format(i, i))

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[124 380   0 124]
[124 380 142 124]
[161 300 320 161]
[161 380 320 161]
[161 380 142 161]
[300 380   0 300]
[300 380 320 300]
[161 300   0 161]
[161 124 142 161]
[161 124   0 161]


In [220]:
hull.simplices

array([[124, 380,   0],
       [124, 380, 142],
       [161, 300, 320],
       [161, 380, 320],
       [161, 380, 142],
       [300, 380,   0],
       [300, 380, 320],
       [161, 300,   0],
       [161, 124, 142],
       [161, 124,   0]], dtype=int32)

In [241]:
[point.sum() for point in points]

[6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6041.684210526316,
 6041.684210526316,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6083.368421052632,
 6083.368421052632,
 6083.368421052632,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6125.0526315789475,
 6125.0526315789475,
 6125.0526315789475,
 6125.0526315789475,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6000.0,
 6166.736842105263,
 6166.736842105263,
 6

In [243]:
points[:]

array([[3270., 2730.,    0.],
       [3270., 2730.,    0.],
       [3270., 2730.,    0.],
       ...,
       [   0., 6792.,    0.],
       [   0., 6792.,    0.],
       [   0., 6792.,    0.]])